In [1]:
from google.colab import drive
drive.mount('/content/myDrive')

Mounted at /content/myDrive


In [2]:
%cd myDrive/MyDrive/hgnn-master

/content/myDrive/.shortcut-targets-by-id/1-_cJVH-_Ijs8tGfwoppAWipbsBTP2zEg/hgnn-master


In [3]:
!pip install scipy==1.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 44.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


In [4]:
import os
import torch

In [5]:
# torch geometric
try:
    import torch_geometric
except ModuleNotFoundError:
    # Installing torch geometric packages with specific CUDA+PyTorch version.
    # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details
    TORCH = '2.0.0'
    CUDA = 'cu' + torch.version.cuda.replace('.','')
    !pip install torch-scatter     -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-sparse      -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-cluster     -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-geometric
    import torch_geometric

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 107.7 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 87.5 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.6 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.9/884.9 kB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 11.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=118e1af74adc8d304b70bd86fd1a78c3e5de2112d2e4ce5c2b51dbee68e55ff9
  Stored in directory: /root/.c

In [6]:
trials = 1
epochs = 100
# model_names = ["GCN", "GC", "RGGC", "GAT", "HGCN", "HGC", "HRGGC", "HGAT"]
num_layers = [4]
dims = [256]
learning_rates = [5e-3]
datasets_dict = {"AirportBrazil": "Brazil"}
# datasets_dict = {"AirportBrazil": "brazil","AirportEurope": "europe", "AirportUSA": "usa", \
#             "CiteSeer": "citeseer", "Cora": "cora", "PubMed": "pubmed", \
#              "WebKB-Cornell": "Cornell", "WebKB-Texas": "Texas", "WebKB-Wisconsin": "wisconsin"}
datasets = ['cora', 'citeseer', 'pubmed', 'usa', 'brazil', 'europe', 'cornell', 'texas', 'Wisconsin']

In [7]:
time_dic = {}

for dataset_key in datasets_dict.keys():
  time_dic[dataset_key] = {}
  time_dic[dataset_key]['HGNN'] = 0

for dataset_key, dataset_val in datasets_dict.items():
    config = 1
    for num_layer in num_layers:
      if dataset_val == "pubmed" and num_layer > 2:
        print("==========")
        print("Skipping PubMed on >2 layer for now...")
        print("==========")
        continue
      for d in dims:
        if dataset_val == "pubmed" and d == 256:
          print("==========")
          print("Skipping PubMed on dim size 256 for now...")
          print("==========")
          continue
        for lr in learning_rates:
          print("==========")
          print("Config", config)
          print("num_layer=" + str(num_layer) + ", d=" + str(d) + ", lr=" + str(lr))
          print("==========")
          # already saved
          #if os.path.exists(f"../HGNNs_Link_Prediction/models_report/{dataset}_HGNN_{num_layer}_{d}_{lr}_{trial}_report.json"):
              # print(f"already saved {dataset}_HGNN_{num_layer}_{d}_{lr}_{trial}_report.json")
              #continue
          print(f"{dataset_val}_HGNN_{num_layer}_{d}_{lr}_1")
          !python -m torch.distributed.launch --nproc_per_node=1 time_main.py --task node_classification --dataset_str $dataset_val --trial_number=1 --gnn_layer=$num_layer --embed_size=$d --lr=$lr --max_epochs=10
          config += 1

Config 1
num_layer=4, d=256, lr=0.005
Brazil_HGNN_4_256_0.005_1
/usr/local/lib/python3.10/dist-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
INFO - 08/21/23 09:01:24 - 0:00:23 - save debug info to log/2023_08_21_09_01_01_804577.log
INFO - 08/21/23 09:01:24 - 0:00:23 - Namespace(name='2023_08_21_09_01_01_804577', task='node_classification', select_manifold='lorentz', seed=1692608461, trial_number=1, world_size=None, local_rank=0, distributed_method='multi_gpu', dataset_str='Brazil', patience=100, lr=0.005, lr_hyperbolic=0.001, optimizer='adam', lr_scheduler='none', hyper_optimizer='ramsgrad', num_cen